<a href="https://colab.research.google.com/github/Minatozaki0911/CapstoneProject202/blob/main/Do_an_HK202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras

In [2]:
import os
import cv2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style

from tensorflow import keras
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import pandas as pd

In [3]:
!unzip '/content/drive/MyDrive/Colab Notebooks/DO_AN_HK202/dataHandLang.zip' -d '/content/drive/mMyDrive/Colab Notebooks/DO_AN_HK202/train'

Archive:  /content/drive/MyDrive/Colab Notebooks/DO_AN_HK202/dataHandLang.zip
checkdir:  cannot create extraction directory: /content/drive/mMyDrive/Colab Notebooks/DO_AN_HK202/train
           No such file or directory


In [4]:
gesture_names = {0: 'E',
                 1: 'L',
                 2: 'F',
                 3: 'V',
                 4: 'B'}

gestures = {'L_': 'L',
           'fi': 'E',
           'ok': 'F',
           'pe': 'V',
           'pa': 'B'
            }

gestures_map = {'E': 0,
                'L': 1,
                'F': 2,
                'V': 3,
                'B': 4
                }
dimension = 224            

In [5]:
image_path = '/content/drive/MyDrive/Colab Notebooks/DO_AN_HK202/train'
model_path = '/content/drive/MyDrive/Colab Notebooks/DO_AN_HK202/'

In [6]:
def process_image(path):
    img = Image.open(path)
    img = img.resize((dimension, dimension))
    img = np.array(img)
    return img

In [7]:
def process_data(X_data, y_data):
    X_data = np.array(X_data, dtype = 'float32')
    X_data = np.stack((X_data,)*3, axis=-1)
    X_data /= 255
    y_data = np.array(y_data)
    y_data = to_categorical(y_data)
    return X_data, y_data

In [8]:
def walk_file_tree(image_path):
    X_data = []
    y_data = []
    for directory, subdirectories, files in os.walk(image_path):
        for file in files:
            if not file.startswith('.'):
                path = os.path.join(directory, file)
                gesture_name = gestures[file[0:2]]
                print(gesture_name)
                print(gestures_map[gesture_name])
                y_data.append(gestures_map[gesture_name])
                X_data.append(process_image(path))

            else:
                continue

    X_data, y_data = process_data(X_data, y_data)
    return X_data, y_data

In [ ]:
X_data, y_data = walk_file_tree(image_path)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,
                                                    test_size = 0.2, 
                                                    random_state=12, 
                                                    stratify=y_data)

In [11]:
model_checkpoint = ModelCheckpoint(filepath=model_path, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_accuracy',
                               min_delta=0,
                               patience=10,
                               verbose=1,
                               mode='auto',
                               restore_best_weights=True)

In [12]:
model1 = VGG16(weights='imagenet', include_top=False, input_shape=(dimension, dimension, 3))
optimizer1 = optimizers.Adam()
base_model = model1

In [13]:
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

In [14]:
model = Model(inputs=base_model.input, outputs=predictions)

In [15]:
for layer in base_model.layers:
    layer.trainable = False

In [16]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train, epochs=7, batch_size=64, validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stopping, model_checkpoint])

Epoch 1/7
35/35 [==============================] - 775s 22s/step - loss: 0.7383 - accuracy: 0.7147 - val_loss: 0.1118 - val_accuracy: 0.9673
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/DO_AN_HK202/assets
Epoch 2/7
35/35 [==============================] - 773s 22s/step - loss: 0.0821 - accuracy: 0.9745 - val_loss: 0.0463 - val_accuracy: 0.9855
INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab Notebooks/DO_AN_HK202/assets
Epoch 3/7
35/35 [==============================] - 775s 22s/step - loss: 0.0290 - accuracy: 0.9900 - val_loss: 0.0753 - val_accuracy: 0.9782
Epoch 4/7
35/35 [==============================] - 773s 22s/step - loss: 0.0201 - accuracy: 0.9936 - val_loss: 0.1026 - val_accuracy: 0.9709
Epoch 5/7
35/35 [==============================] - 772s 22s/step - loss: 0.0243 - accuracy: 0.9927 - val_loss: 0.0465 - val_accuracy: 0.9891
Epoch 6/7
35/35 [==============================] - 772s 22s/step - loss: 0.0041 - accuracy: 0.9991 - val_loss

In [18]:
model.save('ThaiBaoKhangModel.h5')